In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from medplexity.benchmarks.pubmedqa.pubmedqa_dataset_builder import PubmedQADatasetBuilder
from medplexity.benchmarks.pubmedqa.pubmedqa_prompt_template import PubmedQAPromptTemplate


In [3]:
# Set your OpenAI API key
OPENAI_API_KEY = ""

In [4]:
dataset = PubmedQADatasetBuilder().build_dataset()

In [5]:
from medplexity.benchmarks.pubmedqa.pubmedqa_dataset_builder import PubmedQAInput


def input_adapter(pubmedqa_input: PubmedQAInput):
    prompt_template = PubmedQAPromptTemplate()

    return prompt_template.format(
        question=pubmedqa_input.question,
        context=pubmedqa_input.contexts[0]
    )

In [6]:
from medplexity.benchmarks.pubmedqa.pubmedqa_prompt_template import AnswerWithExplanation

def output_adapter(output_json: str) -> AnswerWithExplanation:
    parsed_output = AnswerWithExplanation.model_validate_json(output_json)

    return parsed_output

In [7]:
from medplexity.llms.openai_caller import OpenAI
from medplexity.chains.evaluation_adapter_chain import EvaluationAdapterChain

chain = EvaluationAdapterChain(
    llm=OpenAI(
        api_token=OPENAI_API_KEY
    ),
    input_adapter=input_adapter,
    output_adapter=output_adapter,
)

In [8]:
input_adapter(dataset[0].input)

'The following are multiple choice questions about medical knowledge. Solve them in a step-by-step fashion, starting by summarizing the available information. Output a JSON with the answer (give back only the letter) and an explanation for it.\nContext: To describe the interstitial fluid (ISF) and plasma pharmacokinetics of meropenem in patients on continuous venovenous haemodiafiltration (CVVHDF). This was a prospective observational pharmacokinetic study. Meropenem (500 mg) was administered every 8 h. CVVHDF was targeted as a 2-3 L/h exchange using a polyacrylonitrile filter with a surface area of 1.05 m2 and a blood flow rate of 200 mL/min. Serial blood (pre- and post-filter), filtrate/dialysate and ISF concentrations were measured on 2 days of treatment (Profiles A and B). Subcutaneous tissue ISF concentrations were determined using microdialysis. A total of 384 samples were collected. During Profile A, the comparative median (IQR) ISF and plasma peak concentrations were 13.6 (12.0

In [9]:
def comparator(expected_output: str, predicted_output: AnswerWithExplanation):

    return expected_output == predicted_output.answer

In [10]:
from medplexity.evaluators.sequential_evaluator import SequentialEvaluator

evaluator = SequentialEvaluator(
    chain=chain,
    comparator=comparator
)

In [11]:
dataset[0].input

PubmedQAInput(question='Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?', contexts=['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.', 'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (E

In [12]:
evaluation = evaluator.evaluate(dataset[:2])

100%|██████████| 2/2 [00:07<00:00,  3.96s/it]


In [13]:
correct, incorrect = evaluation.partition_by_correctness()

In [14]:
evaluation.accuracy()

0.5

In [15]:
print(evaluation.evaluation_results[0].model_dump_json(indent=4))

{
    "input": {
        "question": "Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?",
        "contexts": [
            "Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.",
            "The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not u

In [16]:
print(incorrect[0].model_dump_json(indent=4))

{
    "input": {
        "question": "Landolt C and snellen e acuity: differences in strabismus amblyopia?",
        "contexts": [
            "Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical details appear under the same visual angle. Since optotypes are evaluated on individuals with good visual acuity and without eye disorders, differences in the lower visual acuity range cannot be excluded. In this study, visual acuity measured with the Snellen E was compared to the Landolt C acuity.",
            "100 patients (age 8 - 90 years, median 60.5 years) with various eye disorders, among them 39 with amblyopia due to strabismus, and 13 healthy volunteers were tested. Charts with the Snellen E and the Landolt C (Precision Vision) which mimic the ETDRS charts were used to assess visual acuity. Three out of 5 optotypes per line had to be correctly identified, while wrong answers were monito